This notebook is the one I used to test the port from the original repository's code into something I could use. As such, it uses the standard chair data and configuration settings.

# Train

In [1]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
from torch.autograd import Variable
from pcgan.datasets import PartDataset
from pcgan.pointnet.descriminators import (PointNetCls,
                                           PointDecoder,
                                           PointNetAE,
                                           PointNetReg,
                                           PointNetReg2,
                                           PointNetDenseCls)
from pcgan.pointnet.generators import (PointGen,
                                       PointGenComp,
                                       PointGenComp2,
                                       PointGenR,
                                       PointGenR2,
                                       PointGenR3,
                                       PointGenC,
                                       PointGenPSG)

In [2]:
batchSize = 128
nepoch = 2
num_points = 50
#manualSeed = random.randint(1, 10000)
manualSeed = 3329
print('Seed is %i' % manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

workers = 32       # For data loading
outf = 'models'    # Output folder

data_folder = 'data'
classes = ['H_relaxed', 'H_unrelaxed', 'CO_relaxed', 'CO_unrelaxed']

Seed is 3329


In [3]:
dataset = PartDataset(root=data_folder,
                      class_choice=classes,
                      classification=True)
dataloader = torch.utils.data.DataLoader(dataset,
                                         batch_size=batchSize,
                                         shuffle=True,
                                         num_workers=workers)

test_dataset = PartDataset(root=data_folder,
                           class_choice=classes,
                           classification=True,
                           train=False)
testdataloader = torch.utils.data.DataLoader(test_dataset,
                                             batch_size=batchSize,
                                             shuffle=True,
                                             num_workers=workers)

cudnn.benchmark = True

print('Training size of %i' % len(dataset))
print('Test size of %i' % len(test_dataset))
print('%i class[es]' % len(dataset.classes))

Training size of 78450
Test size of 8718
4 class[es]


In [4]:
try:
    os.makedirs(outf)
except OSError:
    pass

classifier = PointNetCls(num_points=num_points)

# Error slowly crept up
gen = PointGen(num_points=num_points)

# Error went up pretty fast wrt epochs
#gen = PointGenPSG(num_points=num_points)

# Error went up very fast
#gen = PointGenC(num_points=num_points)
#gen = PointGenR2(num_points=num_points)

# Dimensionality issues
#gen = PointGenR(num_points=num_points)
#gen = PointGenR3(num_points=num_points)
#gen = PointGenComp(num_points=num_points)
#gen = PointGenComp2(num_points=num_points)

print(classifier)
print(gen)

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1: 
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1: 
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

classifier.apply(weights_init)
gen.apply(weights_init)

classifier.cuda()
gen.cuda()

optimizerD = optim.Adagrad(classifier.parameters(), lr=0.001)
optimizerG = optim.Adagrad(gen.parameters(), lr=0.001)

num_batch = len(dataset)/batchSize

PointNetCls(
  (feat): PointNetfeat(
    (stn): STN3d(
      (conv1): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
      (conv2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
      (conv3): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
      (fc1): Linear(in_features=1024, out_features=512, bias=True)
      (fc2): Linear(in_features=512, out_features=256, bias=True)
      (fc3): Linear(in_features=256, out_features=9, bias=True)
      (relu): ReLU()
    )
    (conv1): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
    (conv2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
    (conv3): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
    (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn3): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (fc1): Linear(in_features=1024, out_features=512, bias=True)
  (fc2): Lin

In [ ]:
%matplotlib notebook
fig = plt.figure()
ax = fig.add_subplot(111)
plt.ion()
plt.legend()
fig.subplots_adjust(bottom=0.3)
fig.show()
fig.canvas.draw()

d_losses = []
g_losses = []

# Asymptotic generator performance within [1, 5], discriminator within [0, 0.7]
loss_fun = F.nll_loss
#loss_fun = F.cross_entropy

# Did not run
#loss_fun = F.poisson_nll_loss
#loss_fun = F.kl_div
#loss_fun = F.binary_cross_entropy
#loss_fun = F.binary_cross_entropy_with_logits
#loss_fun = F.smooth_l1_loss
#loss_fun = F.l1_loss
#loss_fun = F.mse_loss
#loss_fun = F.soft_margin_loss

for epoch in range(nepoch):
    for i, data in enumerate(dataloader): 
        optimizerD.zero_grad()
        points, _ = data
        points = Variable(points)

        bs = points.size()[0]
        target = Variable(torch.from_numpy(np.ones(bs,).astype(np.int64))).cuda()
        points = points.transpose(2,1)
        points = points.cuda()

        pred, trans = classifier(points)
        loss1 = loss_fun(pred, target)

        sim_noise = Variable(torch.randn(bs, 100)).cuda()
        fake = gen(sim_noise)
        fake_target = Variable(torch.from_numpy(np.zeros(bs,).astype(np.int64))).cuda()
        pred2, trans2 = classifier(fake)

        loss2 = loss_fun(pred2, fake_target)
        lossD = (loss1 + loss2)/2
        lossD.backward()

        optimizerD.step()
        optimizerG.zero_grad()

        sim_noise = Variable(torch.randn(bs, 100)).cuda()
        points = gen(sim_noise)
        pred, trans = classifier(points)
        target = Variable(torch.from_numpy(np.ones(bs,).astype(np.int64))).cuda()
        lossG = loss_fun(pred, target)
        lossG.backward()
        optimizerG.step()

        d_losses.append(lossD.data[0])
        g_losses.append(lossG.data[0])
        ax.clear()
        ax.plot(d_losses, label='Descriminator')
        ax.plot(g_losses, label='Generator')
        fig.canvas.draw()
        fig.canvas.flush_events()

    torch.save(classifier.state_dict(), '%s/modelD_%d.pth' % (outf, epoch))
    torch.save(gen.state_dict(), '%s/modelG_%d.pth' % (outf, epoch))

# Generate

In [20]:
gen = PointGen(num_points=num_points)
gen.load_state_dict(torch.load('nll_models/modelG_24.pth'))
gen.cuda()

PointGen(
  (fc1): Linear(in_features=100, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=1024, bias=True)
  (fc4): Linear(in_features=1024, out_features=150, bias=True)
  (th): Tanh()
)

In [24]:
i_ = 50
j_ = 3
k_ = 100

sim_noise = Variable(torch.randn(j_, k_)).cuda()
sim_noises = Variable(torch.zeros(i_*j_, k_)).cuda()

for j in range(j_):
    for i in range(i_):
        x = (1-i/i_)
        sim_noises[i + i_*j] = (sim_noise[j]*x + 
                                sim_noise[(j+1) % j_] * (1-x))

points_raw = gen(sim_noises)
points = points.cpu().transpose(2, 1).data.numpy()
print(points.shape)

(150, 50, 3)


# View
Let's test MY viewing method

In [22]:
import ase
from ase.visualize import view

In [42]:
expansion_factor = 25
atom_types = ['K', 'S', 'Mn', 'Li', 'Ac']
atom_symbols = np.random.choice(atom_types, size=n_atoms)

image_number = random.randint(0, points.shape[0])
print('Randomly selected cloud number %i' % image_number)

positions = points[image_number, :, :] * expansion_factor
n_atoms = positions.shape[0]
atoms = ase.Atoms(atom_symbols, positions)
view(atoms, viewer='x3d')

Randomly selected cloud number 148


In [49]:
atoms.write('entropy.traj')